# Программирование на Python
## Семинар 7. Python для анализа данных
**Data source:** https://www.kaggle.com/datasets/spscientist/students-performance-in-exams.

**Columns:**
- `gender` (binary variable);
- `race/ethnicity` (categorical variable);
- `parental level of education` (categorical variable, student's parents' level of education);
- `lunch` (categorical variable, quality of student's lunch);
- `test preparation course` (categorical variable, whether student managed to complete preparation course for the test);
- `math score` (numeric variable, score for Math test);
- `reading score` (numeric variable, score for Reading part);
- `writing score` (numeric variable, score for Writing part).

#### Задание 1
Загрузите файл StudentsPerformance.csv используя модуль `csv` и возможности модуля `collections`. Не забудьте о том, что для удобства числовые типы данных будет удобнее привести сразу. На выходе должен получиться словарь с ключами-названиями колонок и значениями-колонками.

In [9]:
import csv
from collections import defaultdict

path = '../Занятие 1/StudentsPerformance.csv'
int_cols = ['math score', 'reading score', 'writing score']
data = defaultdict(list)

with open(path, mode='r', newline='') as file:
    csvfile = csv.DictReader(file, delimiter=',')

    for dct in csvfile:
        for key, value in dct.items():
            if key in int_cols:
                data[key].append(int(value))
            else:
                data[key].append(value)

#### Задание 2
Напишите функцию `show`, которая будет выводить название и первые `n` значений каждой колонки в виде таблицы.

In [ ]:
# наш код здесь

#### Задание 3
Подготовка и анализ данных почти всегда подразумевают изпользование таких pipeline'ов, как "group->aggregate" и "group->apply". Имеется в виду, что значения переменной X делятся на группы по значениям переменной Y, и над этими группами производятся различные операции. Реализуйте следующие функции:

- `group`: принимает на вход векторы X и Y и возвращает словарь, в котором ключами являются уникальные значения переменной Y, а значениями - списки соответствующих значений переменной X;
- `aggregate`: принимает на вход словарь, возвращаемый функцией `group`, а также функцию `fun`, которая должна быть применена к каждому его значению. Возвращаться должен словарь, состоящий из тех же ключей и значений-результатов применения функции `fun`.

In [13]:
print(data['math score'][:10])

[72, 69, 90, 47, 76, 71, 88, 40, 64, 38]


In [14]:
print(data['test preparation course'][:10])

['none', 'completed', 'none', 'none', 'none', 'none', 'completed', 'none', 'completed', 'none']


In [19]:
from statistics import mean, median

def group(vector, grouping):
    grouped = defaultdict(list)

    for value, key in zip(vector, grouping):
        grouped[key].append(value)

    return grouped

def aggregate(grouped, fun):
    aggregated = {key: fun(value) for key, value in grouped.items()}

    return aggregated

In [ ]:
# H: те, кто готовится, сдает математику лучше

# переменные: 1) готовился или нет, 2) балл по математике

# как будем проверять:
# 1) разбить на группы по типу подготовки к тесту
# 2) агрегировать значения балла по математике в этих группах (в данном случае mean)

In [21]:
math_score_grouped = group(data['math score'], data['test preparation course'])
aggregate(math_score_grouped, mean)

{'none': 64.0778816199377, 'completed': 69.69553072625699}

In [22]:
aggregate(math_score_grouped, max)

{'none': 100, 'completed': 100}

In [23]:
aggregate(math_score_grouped, median)

{'none': 64.0, 'completed': 69.0}

In [17]:
group(data['math score'][:10], data['test preparation course'][:10])

defaultdict(list,
            {'none': [72, 90, 47, 76, 71, 40, 38], 'completed': [69, 88, 64]})

#### Задание 4
Выдвините гипотезы о взаимодействии числовой и категориальной переменных и протестируйте их используя подготовленные вами функции.

In [ ]:
# H1: баллы по чтению положительно зависят от уровня образования родителей

# reading score               -> numeric
# parental level of education -> categorical

In [25]:
from collections import Counter

Counter(data['parental level of education'])

Counter({'some college': 226,
         "associate's degree": 222,
         'high school': 196,
         'some high school': 179,
         "bachelor's degree": 118,
         "master's degree": 59})

In [26]:
reading_score_grouped = group(data['reading score'], data['parental level of education'])
aggregate(reading_score_grouped, mean)

{"bachelor's degree": 73,
 'some college': 69.46017699115045,
 "master's degree": 75.37288135593221,
 "associate's degree": 70.92792792792793,
 'high school': 64.70408163265306,
 'some high school': 66.93854748603351}

In [27]:
writing_score_grouped = group(data['writing score'], data['parental level of education'])
aggregate(writing_score_grouped, mean)

{"bachelor's degree": 73.38135593220339,
 'some college': 68.84070796460178,
 "master's degree": 75.67796610169492,
 "associate's degree": 69.8963963963964,
 'high school': 62.44897959183673,
 'some high school': 64.88826815642459}

In [28]:
math_score_grouped = group(data['math score'], data['parental level of education'])
aggregate(math_score_grouped, mean)

{"bachelor's degree": 69.38983050847457,
 'some college': 67.1283185840708,
 "master's degree": 69.7457627118644,
 "associate's degree": 67.88288288288288,
 'high school': 62.13775510204081,
 'some high school': 63.497206703910614}

In [ ]:
# H2: баллы по раззным предметам выше у тех, кто лучше ест

# _ score               -> numeric
# lunch                 -> categorical

In [30]:
Counter(data['lunch'])

Counter({'standard': 645, 'free/reduced': 355})

In [36]:
for col in int_cols:
    grouped = group(data[col], data['lunch'])
    aggregated = aggregate(grouped, mean)
    
    print(col, end=': ')
    print({key: round(value, 2) for key, value in aggregated.items()}, end='\n\n')

math score: {'standard': 70.03, 'free/reduced': 58.92}

reading score: {'standard': 71.65, 'free/reduced': 64.65}

writing score: {'standard': 70.82, 'free/reduced': 63.02}

